# List of Exposures with Hologram at USDF


- work with Weakly_2023_01
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/09/19
- update : 2023/07/17



In [1]:
import sys
print(sys.executable)

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-6.0.0/bin/python3


In [2]:
print(sys.path)

['/sdf/data/rubin/user/dagoret/AuxTelComm/notebooks_usdf/butlertools', '/home/d/dagoret/repos/repos_w_2023_27/atmospec/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/ctrl_mpexec/g88183d3282+88d805ed8c/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/spectractor/g7bf34a7a6e/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/obs_lsst/gfd7e96728c+6f6aa72e7b/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/pipe_tasks/gfb267207ab+bb26f3254a/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/shapelet/gd4112516d0+5ce2cc5508/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/meas_modelfit/g807e3ba568+b7d80ac9cf/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/scarlet_extensions/g9d18589735+d445b4966c/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-6.0.0/Linux64/proxmin/g33b4157f25/l

In [3]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2023_27/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-6.0.0/eups 	setup


In [4]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

w_2023_27
   gdf42428520+4e9f6d16c8 	current w_2023_27 setup


In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits
from astropy.time import Time

In [6]:
from IPython.display import display, HTML

In [7]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [8]:
import lsst.daf.butler as dafButler

In [9]:
#repo = "/sdf/group/rubin/repo/main"
repo="/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [10]:
collection='LATISS/raw/all'

## List of Exposures

In [11]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num','time_start','time_end' ,'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle','science_program','jd','mjd'])

In [12]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= "instrument='LATISS'")):
    
    
    #df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,info.timespan.begin.to_string(),info.timespan.end.to_string() ,info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
    #                         info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle, info.science_program]
    df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,pd.to_datetime(info.timespan.begin.to_string()),pd.to_datetime(info.timespan.end.to_string()) ,info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle, info.science_program,
                             info.timespan.begin.jd,info.timespan.begin.mjd ]
    
    if count < 5:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
        
        mjd = Time(info.timespan.begin.to_string()).mjd
        jd = Time(info.timespan.begin.to_string()).jd
        print(mjd,jd)
    

----------------------------------------------------- 0 ---------------------------------------------------------
exposure:
  instrument: 'LATISS'
  id: 2022083100004
  physical_filter: 'unknown~unknown'
  obs_id: 'AT_O_20220831_000004'
  exposure_time: 0.0
  dark_time: 0.0160482
  observation_type: 'bias'
  observation_reason: 'bias'
  day_obs: 20220831
  seq_num: 4
  seq_start: 4
  seq_end: 4
  group_name: '2022083100004'
  group_id: 2022083100004
  target_name: 'UNKNOWN'
  science_program: 'unknown'
  tracking_ra: None
  tracking_dec: None
  sky_angle: None
  azimuth: None
  zenith_angle: None
  has_simulated: False
  timespan: Timespan(begin=astropy.time.Time('2022-08-31 22:58:13.633984', scale='tai', format='iso'), end=astropy.time.Time('2022-08-31 22:58:13.650000', scale='tai', format='iso'))
	 id:                   2022083100004
	 day_obs:              20220831
	 seq_num:              4
	 type-of-observation:  bias
	 target:               UNKNOWN
59822.957102245186 2459823.45710

In [13]:
info.toDict()

{'instrument': 'LATISS',
 'id': 2023071400044,
 'physical_filter': 'SDSSg_65mm~empty',
 'obs_id': 'AT_O_20230714_000044',
 'exposure_time': 30.0,
 'dark_time': 30.0113,
 'observation_type': 'dark',
 'observation_reason': 'dark',
 'day_obs': 20230714,
 'seq_num': 44,
 'seq_start': 44,
 'seq_end': 44,
 'group_name': '2023-07-14T23:17:48.888',
 'group_id': 2693062688880000,
 'target_name': 'azel_target',
 'science_program': 'unknown',
 'tracking_ra': 136.73341812917073,
 'tracking_dec': -20.140960110293687,
 'sky_angle': None,
 'azimuth': 0.479898695652049,
 'zenith_angle': 10.00816161155899,
 'has_simulated': False,
 'timespan': Timespan(begin=astropy.time.Time('2023-07-14 23:17:48.967983', scale='tai', format='iso'), end=astropy.time.Time('2023-07-14 23:18:18.979000', scale='tai', format='iso'))}

In [14]:
df_exposure

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program,jd,mjd
0,2022083100004,AT_O_20220831_000004,20220831,4,2022-08-31 22:58:13.633984,2022-08-31 22:58:13.650,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957102
1,2022083100008,AT_O_20220831_000008,20220831,8,2022-08-31 22:59:16.387013,2022-08-31 22:59:16.398,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957829
2,2022083100007,AT_O_20220831_000007,20220831,7,2022-08-31 22:59:11.978990,2022-08-31 22:59:11.991,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957778
3,2022083100009,AT_O_20220831_000009,20220831,9,2022-08-31 22:59:20.801996,2022-08-31 22:59:20.813,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957880
4,2022083100012,AT_O_20220831_000012,20220831,12,2022-08-31 23:00:00.113994,2022-08-31 23:00:03.124,dark,UNKNOWN,unknown~unknown,NaN,3.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.958335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50945,2023071400040,AT_O_20230714_000040,20230714,40,2023-07-14 23:15:38.984983,2023-07-14 23:16:08.994,dark,azel_target,SDSSg_65mm~empty,10.008162,30.0,136.733418,-20.14096,NaN,unknown,2.460140e+06,60139.969201
50946,2023071400041,AT_O_20230714_000041,20230714,41,2023-07-14 23:16:11.479999,2023-07-14 23:16:41.490,dark,azel_target,SDSSg_65mm~empty,10.008162,30.0,136.733418,-20.14096,NaN,unknown,2.460140e+06,60139.969577
50947,2023071400042,AT_O_20230714_000042,20230714,42,2023-07-14 23:16:43.975014,2023-07-14 23:17:13.985,dark,azel_target,SDSSg_65mm~empty,10.008162,30.0,136.733418,-20.14096,NaN,unknown,2.460140e+06,60139.969953
50948,2023071400043,AT_O_20230714_000043,20230714,43,2023-07-14 23:17:16.470996,2023-07-14 23:17:46.482,dark,azel_target,SDSSg_65mm~empty,10.008162,30.0,136.733418,-20.14096,NaN,unknown,2.460140e+06,60139.970330


In [15]:
df_exposure = df_exposure.astype({"id": int,'day_obs': int,'seq_num':int})

In [16]:
df_exposure

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program,jd,mjd
0,2022083100004,AT_O_20220831_000004,20220831,4,2022-08-31 22:58:13.633984,2022-08-31 22:58:13.650,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957102
1,2022083100008,AT_O_20220831_000008,20220831,8,2022-08-31 22:59:16.387013,2022-08-31 22:59:16.398,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957829
2,2022083100007,AT_O_20220831_000007,20220831,7,2022-08-31 22:59:11.978990,2022-08-31 22:59:11.991,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957778
3,2022083100009,AT_O_20220831_000009,20220831,9,2022-08-31 22:59:20.801996,2022-08-31 22:59:20.813,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.957880
4,2022083100012,AT_O_20220831_000012,20220831,12,2022-08-31 23:00:00.113994,2022-08-31 23:00:03.124,dark,UNKNOWN,unknown~unknown,NaN,3.0,NaN,NaN,NaN,unknown,2.459823e+06,59822.958335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50945,2023071400040,AT_O_20230714_000040,20230714,40,2023-07-14 23:15:38.984983,2023-07-14 23:16:08.994,dark,azel_target,SDSSg_65mm~empty,10.008162,30.0,136.733418,-20.14096,NaN,unknown,2.460140e+06,60139.969201
50946,2023071400041,AT_O_20230714_000041,20230714,41,2023-07-14 23:16:11.479999,2023-07-14 23:16:41.490,dark,azel_target,SDSSg_65mm~empty,10.008162,30.0,136.733418,-20.14096,NaN,unknown,2.460140e+06,60139.969577
50947,2023071400042,AT_O_20230714_000042,20230714,42,2023-07-14 23:16:43.975014,2023-07-14 23:17:13.985,dark,azel_target,SDSSg_65mm~empty,10.008162,30.0,136.733418,-20.14096,NaN,unknown,2.460140e+06,60139.969953
50948,2023071400043,AT_O_20230714_000043,20230714,43,2023-07-14 23:17:16.470996,2023-07-14 23:17:46.482,dark,azel_target,SDSSg_65mm~empty,10.008162,30.0,136.733418,-20.14096,NaN,unknown,2.460140e+06,60139.970330


## Selection of science

In [17]:
df_science = df_exposure[df_exposure.type == 'science']

In [18]:
df_science.sort_values(by="id",ascending=True,inplace=True)

/tmp/ipykernel_5950/3986782663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science.sort_values(by="id",ascending=True,inplace=True)


In [19]:
df_science.tail(60)

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program,jd,mjd
50213,2023070600518,AT_O_20230706_000518,20230706,518,2023-07-07 05:03:35.550001,2023-07-07 05:04:05.794,science,Photo1800-1_026,SDSSr_65mm~empty,17.928725,30.0,270.165687,-24.956565,359.999271,AUXTEL_PHOTO_IMAGING,2.460133e+06,60132.210828
50220,2023070600519,AT_O_20230706_000519,20230706,519,2023-07-07 05:04:23.395004,2023-07-07 05:04:53.633,science,Photo1800-1_027,SDSSr_65mm~empty,18.080685,30.0,270.181970,-24.956561,359.999220,AUXTEL_PHOTO_IMAGING,2.460133e+06,60132.211382
50226,2023070600520,AT_O_20230706_000520,20230706,520,2023-07-07 05:05:07.733005,2023-07-07 05:05:37.974,science,Photo1800-1_027,SDSSg_65mm~empty,18.239400,30.0,270.182005,-24.956577,359.999098,AUXTEL_PHOTO_IMAGING,2.460133e+06,60132.211895
50232,2023070600521,AT_O_20230706_000521,20230706,521,2023-07-07 05:05:52.096997,2023-07-07 05:06:22.329,science,Photo1800-1_027,SDSSi_65mm~empty,18.394836,30.0,270.181908,-24.956528,359.999309,AUXTEL_PHOTO_IMAGING,2.460133e+06,60132.212409
50242,2023070600522,AT_O_20230706_000522,20230706,522,2023-07-07 05:06:59.844005,2023-07-07 05:07:30.083,science,Photo1600-1_007,SDSSi_65mm~empty,44.842695,30.0,240.046865,-24.045367,359.995089,AUXTEL_PHOTO_IMAGING,2.460133e+06,60132.213193
50249,2023070600523,AT_O_20230706_000523,20230706,523,2023-07-07 05:07:44.257000,2023-07-07 05:08:14.501,science,Photo1600-1_007,SDSSg_65mm~empty,45.000865,30.0,240.046903,-24.045261,359.995067,AUXTEL_PHOTO_IMAGING,2.460133e+06,60132.213707
50256,2023070600524,AT_O_20230706_000524,20230706,524,2023-07-07 05:08:28.674020,2023-07-07 05:08:58.920,science,Photo1600-1_007,SDSSr_65mm~empty,45.159036,30.0,240.046866,-24.045293,359.995067,AUXTEL_PHOTO_IMAGING,2.460133e+06,60132.214221
50303,2023070600530,AT_O_20230706_000530,20230706,530,2023-07-07 05:14:10.054988,2023-07-07 05:14:40.296,science,Photo1600-1_017,SDSSr_65mm~empty,46.491133,30.0,239.949701,-23.986042,359.994716,AUXTEL_PHOTO_IMAGING,2.460133e+06,60132.218172
50309,2023070600531,AT_O_20230706_000531,20230706,531,2023-07-07 05:14:54.406991,2023-07-07 05:15:24.637,science,Photo1600-1_017,SDSSg_65mm~empty,46.649055,30.0,239.949791,-23.985973,359.994672,AUXTEL_PHOTO_IMAGING,2.460133e+06,60132.218685
50316,2023070600532,AT_O_20230706_000532,20230706,532,2023-07-07 05:15:38.742015,2023-07-07 05:16:08.993,science,Photo1600-1_017,SDSSi_65mm~empty,46.810553,30.0,239.949814,-23.986064,359.994441,AUXTEL_PHOTO_IMAGING,2.460133e+06,60132.219198


In [20]:
df_science.day_obs.unique()

array([20220912, 20220913, 20220914, 20220927, 20220928, 20220929,
       20221011, 20221012, 20221013, 20221025, 20221026, 20221027,
       20221108, 20221109, 20221110, 20221122, 20221123, 20221124,
       20221207, 20221208, 20221209, 20221210, 20221212, 20230117,
       20230118, 20230119, 20230131, 20230201, 20230202, 20230214,
       20230215, 20230216, 20230228, 20230301, 20230302, 20230314,
       20230315, 20230316, 20230509, 20230510, 20230511, 20230523,
       20230524, 20230525, 20230704, 20230705, 20230706])

## Selection of date

2021-02-16  2021-02-18  2021-06-08  2021-06-10  2021-07-07  2021-09-07  2021-09-09  2021-10-06  2021-11-02  2021-11-04  2022-02-16  2022-03-15  2022-03-17  20220502  
2021-02-17  2021-03-11  2021-06-09  2021-07-06  2021-07-08  2021-09-08  2021-10-05  2021-10-07  2021-11-03  2022-02-15  2022-02-17  2022-03-16  2022-03-18  20220503 20220524 20220607 20220608 20220609

### Add those

20220504 - Not existing

In [21]:
#date_sel = 20220630
#date_sel = 20220630
#date_sel = 20221208
#date_sel = 20221209

# data in january
#date_sel= 20230118
#date_sel= 20230131
#date_sel = 20230201
#date_sel = 20230202
#date_sel = 20230704
#date_sel = 20230705
date_sel = 20230706


In [22]:
df_science_selected = df_science[df_science.day_obs == date_sel ]

In [23]:
len(df_science_selected)

368

# List of filters

In [24]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

array(['empty~holo4_003', 'empty~blue300lpmm_qn1', 'SDSSr_65mm~empty',
       'SDSSg_65mm~empty', 'SDSSi_65mm~empty'], dtype=object)

In [25]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

array(['empty~holo4_003'], dtype='<U15')

## Selection of filter

- But now we have a loop on filters

In [26]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [27]:
pd.set_option('display.max_rows', None)

In [28]:
all_def_science_selected_final = []

for selected_filter in selected_filters:

    print(selected_filter)
    df_science_selected_final = df_science_selected[df_science_selected["filter"] ==  selected_filter]
    
    #print(df_science_selected_final)
    all_def_science_selected_final.append(df_science_selected_final.copy())
    
    df = df_science_selected_final
    df_sel = df[['day_obs', 'seq_num']]
    
    
    NN = len(df)
    
    if NN>0:
        # build the line
        
        df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
    
    
         # prepare output
        filename_out = "visitdispersers_"+ str(date_sel) + "_filt_" + selected_filter.replace('~','-') + ".list"
        print(filename_out)
        
        df_out = df_sel[["line"]]
        df_out.to_csv(filename_out, index=False, header=None)

empty~holo4_003
visitdispersers_20230706_filt_empty-holo4_003.list


/tmp/ipykernel_5950/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)


In [29]:
N = len(all_def_science_selected_final)

In [30]:
if N>=1:
    n0=len(all_def_science_selected_final[0])
    display(all_def_science_selected_final[0])

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,science_program,jd,mjd
50637,2023070600103,AT_O_20230706_000103,20230706,103,2023-07-06 23:06:13.101993,2023-07-06 23:06:43.337,science,HD146233,empty~holo4_003,46.409661,30.0,243.883992,-8.325790,325.869586,spec,2.460132e+06,60131.962652
50638,2023070600104,AT_O_20230706_000104,20230706,104,2023-07-06 23:06:56.015983,2023-07-06 23:07:26.253,science,HD146233,empty~holo4_003,46.262780,30.0,243.883881,-8.325734,325.869500,spec,2.460132e+06,60131.963148
50641,2023070600107,AT_O_20230706_000107,20230706,107,2023-07-06 23:08:46.667002,2023-07-06 23:09:16.901,science,HD146233,empty~holo4_003,45.879727,30.0,243.880001,-8.328276,320.868538,spec,2.460132e+06,60131.964429
50642,2023070600108,AT_O_20230706_000108,20230706,108,2023-07-06 23:09:29.570008,2023-07-06 23:09:59.811,science,HD146233,empty~holo4_003,45.733333,30.0,243.879981,-8.328276,320.868478,spec,2.460132e+06,60131.964926
50653,2023070600118,AT_O_20230706_000118,20230706,118,2023-07-06 23:16:18.055018,2023-07-06 23:16:48.296,science,HD185975,empty~holo4_003,60.531945,30.0,306.903862,-87.421878,343.671664,spec,2.460132e+06,60131.969653
50654,2023070600119,AT_O_20230706_000119,20230706,119,2023-07-06 23:17:01.107008,2023-07-06 23:17:31.344,science,HD185975,empty~holo4_003,60.524014,30.0,306.903542,-87.421875,343.671846,spec,2.460132e+06,60131.970152
50657,2023070600122,AT_O_20230706_000122,20230706,122,2023-07-06 23:18:53.475982,2023-07-06 23:19:23.708,science,HD185975,empty~holo4_003,60.498699,30.0,306.808600,-87.423233,338.670250,spec,2.460132e+06,60131.971452
50658,2023070600123,AT_O_20230706_000123,20230706,123,2023-07-06 23:19:36.377016,2023-07-06 23:20:06.609,science,HD185975,empty~holo4_003,60.490771,30.0,306.808991,-87.423236,338.669770,spec,2.460132e+06,60131.971949
50667,2023070600132,AT_O_20230706_000132,20230706,132,2023-07-06 23:27:11.083019,2023-07-06 23:27:41.327,science,HD146233,empty~holo4_003,42.167178,30.0,243.882117,-8.326990,323.559642,spec,2.460132e+06,60131.977212
50668,2023070600133,AT_O_20230706_000133,20230706,133,2023-07-06 23:27:53.994997,2023-07-06 23:28:24.230,science,HD146233,empty~holo4_003,42.024471,30.0,243.882111,-8.327094,323.559608,spec,2.460132e+06,60131.977708


In [31]:
if N>=2:
    n1=len(all_def_science_selected_final[1])
    display(all_def_science_selected_final[1])

In [32]:
if N>=3:
    n2=len(all_def_science_selected_final[2])
    display(all_def_science_selected_final[2])

In [33]:
if N>=4:
    n3=len(all_def_science_selected_final[3])
    display(all_def_science_selected_final[3])

In [34]:
if N>=5:
    n4=len(all_def_science_selected_final[4])
    display(all_def_science_selected_final[4])

In [35]:
df_forsave = pd.concat(all_def_science_selected_final)

In [36]:
filename_out_csv = "holosummary_"+ str(date_sel) + ".csv"
filename_out_excel = "holosummary_"+ str(date_sel) + ".xlsx"

In [37]:
filename_out_csv

'holosummary_20230706.csv'

In [38]:
df_forsave.to_csv(filename_out_csv) 

In [39]:
! pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable


In [40]:
df_forsave.to_excel(filename_out_excel) 